In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"suyashkumar234","key":"8c583385a33aeacbf173aeb5cc7d4ff7"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d kavyasreeb/hair-type-dataset

Dataset URL: https://www.kaggle.com/datasets/kavyasreeb/hair-type-dataset
License(s): apache-2.0
 92% 161M/175M [00:01<00:00, 187MB/s]
100% 175M/175M [00:01<00:00, 147MB/s]


In [5]:
!unzip hair-type-dataset.zip

Archive:  hair-type-dataset.zip
  inflating: data/Straight/000028d73-Nicole_Scherzinger.jpg  
  inflating: data/Straight/01-10-2019-latest-haircut-for-girls_Equal_Length_4PNG.jpg  
  inflating: data/Straight/01-10-2019-latest-haircut-for-girls_short_pixie_straight_undercut_3PNG.jpg  
  inflating: data/Straight/01b8ef4743e2a5d1b3e888ef18acdfd0.jpg  
  inflating: data/Straight/03e7316208c63582103294f3e1ef8b7b.jpg  
  inflating: data/Straight/051c0f399232bd22fb8ff55a6dde5441.jpg  
  inflating: data/Straight/060c671d89cba47235d3a8a620bed080.jpg  
  inflating: data/Straight/06faceb8ea130fb976a3339f52ba611d--hair-products-for-men-gentlemens-guide.jpg  
  inflating: data/Straight/0a1c060ba84f25dd6f353656818fb646.jpg  
  inflating: data/Straight/0a7bec97b58f02f16f1da241ed4d99dd.jpg  
  inflating: data/Straight/0f9d3172f6ec3206d6f76318830159b2.jpg  
  inflating: data/Straight/1-Saee-Hairstyle-Round-Face-Girls-With-Straightened-Hair.jpg  
  inflating: data/Straight/1-short-spiky-mens-haircut.jpg

In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [7]:
os.makedirs('train/Straight', exist_ok=True)
os.makedirs('train/Wavy', exist_ok=True)
os.makedirs('train/curly', exist_ok=True)
os.makedirs('train/dreadlocks', exist_ok=True)
os.makedirs('train/kinky', exist_ok=True)

os.makedirs('test/Straight', exist_ok=True)
os.makedirs('test/Wavy', exist_ok=True)
os.makedirs('test/curly', exist_ok=True)
os.makedirs('test/dreadlocks', exist_ok=True)
os.makedirs('test/kinky', exist_ok=True)


In [8]:
# Directory where your data is currently stored
base_dir = '/content/data'

# List of updated hair categories
categories = ['Straight', 'Wavy', 'curly', 'dreadlocks', 'kinky']

for category in categories:
    # Get all image files from each hair type category
    category_path = os.path.join(base_dir, category)
    files = [f for f in os.listdir(category_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Check if there are no files in the category
    if len(files) == 0:
        print(f"Warning: No images found in {category} category. Skipping this category.")
        continue

    # Split files into train (80%) and test (20%)
    train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)

    # Move train files
    for file in train_files:
        shutil.move(os.path.join(category_path, file), os.path.join('train', category, file))

    # Move test files
    for file in test_files:
        shutil.move(os.path.join(category_path, file), os.path.join('test', category, file))

print("Train-test split completed!")


Train-test split completed!


In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
import os

Using test data for validation

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values to [0, 1]
    # Data Augmentation on Train dataset
    rotation_range=20,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Randomly zoom in
    horizontal_flip=True,  # Randomly flip images
    fill_mode='nearest'  # Fill pixels that are created during transformations
)

# Create an instance of ImageDataGenerator for validation data
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Only normalization for validation

# Create training data generator
train_generator = train_datagen.flow_from_directory(
    '/content/train',  # Directory with training images
    target_size=(32, 32),  # Resize images
    batch_size=32,  # Number of images to be yielded from the generator per batch
    class_mode='categorical',  # Use 'categorical' for one-hot encoded labels
    color_mode='rgb'  # Change to 'grayscale' if your images are grayscale
)

# Create validation data generator
validation_generator = validation_datagen.flow_from_directory(
    '/content/test',  # Directory with validation images
    target_size=(32, 32),  # Resize images
    batch_size=32,  # Number of images to be yielded from the generator per batch
    class_mode='categorical',  # Use 'categorical' for one-hot encoded labels
    color_mode='rgb'  # Change to 'grayscale' if your images are grayscale
)


Found 1584 images belonging to 5 classes.
Found 399 images belonging to 5 classes.


In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn_model(input_shape=(32, 32, 3), num_classes=5):
    model = models.Sequential()

    # First Convolutional Block
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Second Convolutional Block
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Third Convolutional Block
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flattening the output
    model.add(layers.Flatten())

    # Fully Connected Layer
    model.add(layers.Dense(128, activation='relu'))

    # Output Layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Example usage
model = create_cnn_model()
model.summary()  # Displays the model summary

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 30, 30, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 4, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 159,557 (623.27 KB)

 Trainable params: 159,557 (623.27 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
# Fit the model using the train and validation data
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=500  # Adjust the number of epochs as needed
)

Epoch 1/500


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


49/49 ━━━━━━━━━━━━━━━━━━━━ 19s 249ms/step - accuracy: 0.2393 - loss: 1.5819 - val_accuracy: 0.2656 - val_loss: 1.5688
Epoch 2/500
 1/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2188 - loss: 1.6369

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.2188 - loss: 1.6369 - val_accuracy: 0.0667 - val_loss: 1.5527
Epoch 3/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 10s 188ms/step - accuracy: 0.2614 - loss: 1.5519 - val_accuracy: 0.3177 - val_loss: 1.5094
Epoch 4/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2500 - loss: 1.4711 - val_accuracy: 0.2000 - val_loss: 1.6297
Epoch 5/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 9s 167ms/step - accuracy: 0.3263 - loss: 1.5164 - val_accuracy: 0.3073 - val_loss: 1.4820
Epoch 6/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 911us/step - accuracy: 0.4688 - loss: 1.4136 - val_accuracy: 0.6000 - val_loss: 1.3450
Epoch 7/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 11s 198ms/step - accuracy: 0.3594 - loss: 1.4744 - val_accuracy: 0.3880 - val_loss: 1.4403
Epoch 8/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.3750 - loss: 1.3654 - val_accuracy: 0.2667 - val_loss: 1.2136
Epoch 9/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 19s 180ms/step - accuracy: 0.3871 - loss: 1.4546 - val_accuracy: 0.4297 